# Задание 1
    1. Заснять на веб-камеру или скачать с интернета цветное видео (файл_1) длинной не менее 50 с, в котором человек производит какие-либо движения (допустимы даже минимальные: необходимо, чтобы его лицо не было зафиксировано в одном положении в течение видео) и при этом большую часть времени смотрит в объектив камеры (но не все время, т.е. человек может отворачиваться, посмотреть в другую сторону). Необходимо к каждому кадру видеопотока применять различные преобразования, так что в видео:
    • [0c; 5c): кадр без изменений
    • [6c; 10c): размытие с квадратной матрицей из 1 размером 3, 
    • [10c; 15c): размытие с квадратной матрицей из 1 размером 5, 
    • [15c; 20c): размытие с квадратной матрицей из 1 размером 9. 
    • [20c; 25c): размытие с квадратной матрицей из 1 размером 15. 
    • [25c; 30c): обнулить цветовой канал первый (B)
    • [30c; 35c): обнулить второй цветовой канал (G)
    • [35c; 40c): обнулить второй цветовой канал (R)
    • [40c; 45с): применить эрозию с ядром размером 3
    • [45c; 50с): применить эрозию с ядром размером 5
Записать сохранить модифицированную видеозапись в файл2.

In [2]:
import numpy as np
import math
import cv2
from matplotlib import pyplot as plt
from PIL import Image

In [3]:
fps = 30
cap = cv2.VideoCapture("/home/qwe/Desktop/анализ изображений/test1.mp4")

fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('test2.avi', fourcc, fps, size, isColor=True)

def f2(c, img):
    img[:,:,c] = 0
    return img

In [4]:
def f(i, frame):
    return {
        0: frame.copy(),
        1: cv2.GaussianBlur(frame, (3, 3), 1),
        2: cv2.GaussianBlur(frame, (5, 5), 1),
        3: cv2.GaussianBlur(frame, (9, 9), 1),
        4: cv2.GaussianBlur(frame, (15, 15), 1),
        5: f2(0, frame.copy()),
        6: f2(1, frame.copy()),
        7: f2(2, frame.copy()),
        8: cv2.erode(frame.copy(), cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3)), iterations=1),
        9: cv2.erode(frame.copy(), cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5)), iterations=1)
    }[i]

In [5]:
for i in range(10):
    for j in range(i*5*fps, (i+1)*5*fps):
        # Capture frame by frame
        ret, frame = cap.read()

        res = f(i, frame)
        out.write(res)
# When everything done , release the capture
cap.release()
out.release()
cv2.destroyAllWindows()


# Задание 2 
Написать программу, которая к видео из файла2 применяет признаки Хаара, детектирует лица и обводит их в прямоугольную рамку. Записать полученное видео в файл3. Сделать выводы о том насколько ухудшение качества видеопотока влияет на определение лиц с помощью признаков Хаара.

In [9]:
fps = 30
face_cascade_db = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture("/home/qwe/Desktop/анализ изображений/test2.avi")

fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('test3.avi', fourcc, fps, size, isColor=True)

for i in range(50*fps):
    ret, frame = cap.read()

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade_db.detectMultiScale(frame_gray, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    out.write(frame)
# When everything done , release the capture
cap.release()
out.release()
cv2.destroyAllWindows()